In [2]:
#%pip install -U pandas

In [3]:
#%pip install -U pyLDAvis

In [4]:
## imports
import os, sys
import pprint as pp

In [5]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [6]:
## variables
random_target = False

## HDP: The following parameters need to be relatively large for HDP, unlike LDA
min_bot_size    = 3
term_minfreq    = 3
abuse_threshold = 0.03

## doc
doc_max_size = 30 # max count of words in a sentence
doc_min_size = 5  # min count of words in a sentence

## term
ngram_is_inclusive = True
term_types   = [ 'w1gram', 'w2gram', 'w3gram', 'skippy_w2gram', 'skippy_w3gram' ]
term_type    = term_types[-1]
max_gap_val  = round(doc_max_size * 0.5)
gap_mark     = " … "

In [7]:
## Get target files
import glob
data_dir = "data/Darwin-texts/single-lined/"
target_files = glob.glob(f"{data_dir}/*")
target_files = [ file for file in target_files if ".txt" in file ]
pp.pprint(target_files)

['data/Darwin-texts/single-lined/sl-2485-body.utf-8.txt',
 'data/Darwin-texts/single-lined/sl-1227-body.utf-8.txt']


In [8]:
## Read data from files

import random
import pandas as pd

if random_target:
    file = random.choice(target_files)
else:
    file = target_files[0]
print(f"processing: {file}")

if file.endswith(".csv"):
    with open(file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = ['sentence'])
elif file.endswith(".txt"):
    with open(file, "rt") as f:
        raw_df = pd.read_table(f, encoding = 'utf8', header = None, names = ['sentence'])
elif file.endswith(".xlsx"):
    with open(file, "rb") as f:
        raw_df = pd.read_excel(f, index_col = 0)
# 
raw_df.sample(10)

processing: data/Darwin-texts/single-lined/sl-2485-body.utf-8.txt


,sentence
1660,But he who believes in the slow modification o...
1296,The gain in strength and durability in a tendr...
530,Nevertheless (and this is the remarkable fact)...
1723,Nearly all the species of _Lathyrus_ possesses...
1351,A single delicate touch on the concave surface...
1847,Fritz Müller informs me that he saw in the for...
259,This power would be necessary for any species ...
1736,"Secondly, if a twining plant or a tendril gets..."
1795,Fritz Müller has published some interesting fa...
1607,"When the rootlets were torn from the glass, at..."


In [9]:
## build w1gram
import re
w1grams = raw_df['sentence'].apply(lambda x: re.split(r"\s+", x))

## convert to lowercase
w1grams = [ [ x.lower() for x in w1gram ] for w1gram in w1grams ]

## remove ineffective characters
removed_chars = r"[-.,:;!?()_\"\'“”‘’]"
w1grams = [ [ re.sub(removed_chars, "", x) for x in w1gram ] for w1gram in w1grams ]

## exclude single-character words
w1grams = [ [x for x in w1gram if len(x) > 1 ] for w1gram in w1grams ]

In [10]:
## remove too frequent words
from collections import Counter
all_words = [ ]
[ all_words.extend(x) for x in w1grams ] 
word_counts = Counter(all_words)
reduct_rate = 0.003 # needs to be optimized text-wise
too_frequents = word_counts.most_common(round(len(word_counts) * reduct_rate))
pp.pprint(too_frequents)
len(too_frequents)

[('the', 4019),
 ('of', 1941),
 ('and', 1439),
 ('in', 1360),
 ('to', 1025),
 ('is', 527),
 ('it', 509),
 ('that', 486),
 ('as', 473),
 ('with', 460),
 ('this', 414)]


11

In [11]:
## exclude too frequent words
w1grams = [ [x for x in w1gram if not x in too_frequents ] for w1gram in w1grams ]
raw_df['w1gram'] = w1grams
raw_df['size'] = raw_df['w1gram'].apply(lambda x: len(x))
raw_df

,sentence,w1gram,size
0,The Movement and Habits of Climbing Plants by...,"[the, movement, and, habits, of, climbing, pla...",10
1,PREFACE,[preface],1
2,THIS Essay first appeared in the ninth volume ...,"[this, essay, first, appeared, in, the, ninth,...",18
3,"It is here reproduced in a corrected and, I ho...","[it, is, here, reproduced, in, corrected, and,...",14
4,"The illustrations were drawn by my son, George...","[the, illustrations, were, drawn, by, my, son,...",9
...,...,...,...
1853,"Sachs’ ‘Text-Book of Botany’ 1875, pp. 766, 785.","[sachs, textbook, of, botany, 1875, pp, 766, 785]",8
1854,Fritz Müller also has shown in relation to our...,"[fritz, müller, also, has, shown, in, relation...",43
1855,Mr. Herbert Spencer has recently argued (‘Prin...,"[mr, herbert, spencer, has, recently, argued, ...",30
1856,"Annales des Sc. Nat. 4th series, Bot. tom. vi....","[annales, des, sc, nat, 4th, series, bot, tom,...",11


In [12]:
## define df by filtering by length
print(len(raw_df))
df = raw_df[ (doc_min_size <= raw_df['size']) & (raw_df['size'] <= doc_max_size) ]
print(len(df))

1858
1104


In [13]:
## skippy word 2-grams
import ngrams_skippy
skippy_w2grams = [ ngrams_skippy.gen_skippy2grams(x, sep = " ", missing_mark = gap_mark, max_distance = max_gap_val)
                  for x in df['w1gram'] ]

if ngram_is_inclusive:
    for i, gram in enumerate(skippy_w2grams):
        gram.extend(list(df['w1gram'])[i])

df['skippy_w2gram'] = skippy_w2grams
df

/var/folders/yh/c68mhtcj6yn7qsl201kw_7zw0000gn/T/ipykernel_46455/3212885139.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['skippy_w2gram'] = skippy_w2grams


,sentence,w1gram,size,skippy_w2gram
0,The Movement and Habits of Climbing Plants by...,"[the, movement, and, habits, of, climbing, pla...",10,"[the movement, the … and, the … habits, the … ..."
2,THIS Essay first appeared in the ninth volume ...,"[this, essay, first, appeared, in, the, ninth,...",18,"[this essay, this … first, this … appeared, th..."
3,"It is here reproduced in a corrected and, I ho...","[it, is, here, reproduced, in, corrected, and,...",14,"[it is, it … here, it … reproduced, it … in, i..."
4,"The illustrations were drawn by my son, George...","[the, illustrations, were, drawn, by, my, son,...",9,"[the illustrations, the … were, the … drawn, t..."
7,These memoirs ought to be carefully studied by...,"[these, memoirs, ought, to, be, carefully, stu...",25,"[these memoirs, these … ought, these … to, the..."
...,...,...,...,...
1851,"Quoted by Cohn, in his remarkable memoir, “Con...","[quoted, by, cohn, in, his, remarkable, memoir...",18,"[quoted by, quoted … cohn, quoted … in, quoted..."
1852,"Such slight spontaneous movements, I now find,...","[such, slight, spontaneous, movements, now, fi...",29,"[such slight, such … spontaneous, such … movem..."
1853,"Sachs’ ‘Text-Book of Botany’ 1875, pp. 766, 785.","[sachs, textbook, of, botany, 1875, pp, 766, 785]",8,"[sachs textbook, sachs … of, sachs … botany, s..."
1855,Mr. Herbert Spencer has recently argued (‘Prin...,"[mr, herbert, spencer, has, recently, argued, ...",30,"[mr herbert, mr … spencer, mr … has, mr … rece..."


In [14]:
## skippy word 3-grams
import ngrams_skippy
skippy_w3grams = [ ngrams_skippy.gen_skippy3grams(x, sep = " ", missing_mark = gap_mark, max_distance = max_gap_val)
                  for x in df['w1gram'] ]

if ngram_is_inclusive:
    for i, gram in enumerate(skippy_w3grams):
        gram.extend(list(df['skippy_w2gram'])[i])

df['skippy_w3gram'] = skippy_w3grams
df

/var/folders/yh/c68mhtcj6yn7qsl201kw_7zw0000gn/T/ipykernel_46455/887490813.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['skippy_w3gram'] = skippy_w3grams


,sentence,w1gram,size,skippy_w2gram,skippy_w3gram
0,The Movement and Habits of Climbing Plants by...,"[the, movement, and, habits, of, climbing, pla...",10,"[the movement, the … and, the … habits, the … ...","[the movement and, the movement … habits, the ..."
2,THIS Essay first appeared in the ninth volume ...,"[this, essay, first, appeared, in, the, ninth,...",18,"[this essay, this … first, this … appeared, th...","[this essay first, this essay … appeared, this..."
3,"It is here reproduced in a corrected and, I ho...","[it, is, here, reproduced, in, corrected, and,...",14,"[it is, it … here, it … reproduced, it … in, i...","[it is here, it is … reproduced, it is … in, i..."
4,"The illustrations were drawn by my son, George...","[the, illustrations, were, drawn, by, my, son,...",9,"[the illustrations, the … were, the … drawn, t...","[the illustrations were, the illustrations … d..."
7,These memoirs ought to be carefully studied by...,"[these, memoirs, ought, to, be, carefully, stu...",25,"[these memoirs, these … ought, these … to, the...","[these memoirs ought, these memoirs … to, thes..."
...,...,...,...,...,...
1851,"Quoted by Cohn, in his remarkable memoir, “Con...","[quoted, by, cohn, in, his, remarkable, memoir...",18,"[quoted by, quoted … cohn, quoted … in, quoted...","[quoted by cohn, quoted by … in, quoted by … h..."
1852,"Such slight spontaneous movements, I now find,...","[such, slight, spontaneous, movements, now, fi...",29,"[such slight, such … spontaneous, such … movem...","[such slight spontaneous, such slight … moveme..."
1853,"Sachs’ ‘Text-Book of Botany’ 1875, pp. 766, 785.","[sachs, textbook, of, botany, 1875, pp, 766, 785]",8,"[sachs textbook, sachs … of, sachs … botany, s...","[sachs textbook of, sachs textbook … botany, s..."
1855,Mr. Herbert Spencer has recently argued (‘Prin...,"[mr, herbert, spencer, has, recently, argued, ...",30,"[mr herbert, mr … spencer, mr … has, mr … rece...","[mr herbert spencer, mr herbert … has, mr herb..."


In [15]:
## build doc_dict
doc_dict = { i : x for i, x in enumerate(df['sentence']) }
pp.pprint(random.sample(doc_dict.items(), 5))

[(1015,
  'This may be the more safely inferred, as in young and unhealthy specimens '
  'of other tendril-bearing plants similar rudiments may occasionally be '
  'observed.'),
 (676,
  'I have, however, seen several tendrils of _Bryonia dioica_ interlocked, but '
  'they subsequently released one another.'),
 (511,
  'The tips and the inner surfaces of the hooks now began to swell, and in two '
  'or three days were visibly enlarged.'),
 (94,
  'H. M. May 16, 1st circle was made in 9 5 July 20, 2nd 8 0  21, 3rd 8 5  '
  '_Ceropegia Gardnerii_ (Asclepiadaceæ) moves against the sun.'),
 (941,
  'One of the most remarkable is the _Marcgravia umbellata_, the stem of which '
  'in the tropical forests of South America, as I hear from Mr.')]


/var/folders/yh/c68mhtcj6yn7qsl201kw_7zw0000gn/T/ipykernel_46455/3079036891.py:3: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  pp.pprint(random.sample(doc_dict.items(), 5))


In [16]:
## select bots for DTM
print(f"term_type: {term_type}")
bots = df[term_type]
bots = [ bot for bot in bots if len(bot) > min_bot_size ]
random.sample(bots, 3)

term_type: skippy_w3gram


[['older tendril several',
  'older tendril … weeks',
  'older tendril … after',
  'older tendril … its',
  'older tendril … attachment',
  'older tendril … to',
  'older tendril … wall',
  'older tendril … with',
  'older tendril … the',
  'older tendril … branches',
  'older tendril … thickened',
  'older tendril … and',
  'older tendril … spirally',
  'older tendril … contracted',
  'older … several weeks',
  'older … several … after',
  'older … several … its',
  'older … several … attachment',
  'older … several … to',
  'older … several … wall',
  'older … several … with',
  'older … several … the',
  'older … several … branches',
  'older … several … thickened',
  'older … several … and',
  'older … several … spirally',
  'older … several … contracted',
  'older … weeks after',
  'older … weeks … its',
  'older … weeks … attachment',
  'older … weeks … to',
  'older … weeks … wall',
  'older … weeks … with',
  'older … weeks … the',
  'older … weeks … branches',
  'older … weeks

In [17]:
## build diction, corpus = dtm
from gensim.corpora import Dictionary

## dtm
diction = Dictionary(bots)
print(diction)
diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
print(diction)

## corpus
corpus = [ diction.doc2bow(bot) for bot in bots ]

Dictionary<963540 unique tokens: ['and', 'and habits', 'and habits of', 'and habits … by', 'and habits … charles']...>
Dictionary<25613 unique tokens: ['and … climbing', 'and … climbing plants', 'and … of climbing', 'and … of … by', 'and … of … plants']...>


Run HDP

In [18]:
## HDP (max_n_topics = 150)
import numpy as np
import gensim.models
import pyLDAvis.gensim

hdp_full = gensim.models.HdpModel(corpus, diction, random_state = 1)
vis_data = pyLDAvis.gensim.prepare(hdp_full, corpus, diction)
pyLDAvis.display(vis_data)

In [19]:
## HDP (max_n_topics = 90)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 90
hdp = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics)
vis_data = pyLDAvis.gensim.prepare(hdp, corpus, diction)
pyLDAvis.display(vis_data)

In [20]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	output = f"results/LDAvis/Darwin-HDP-max_ntop{max_n_topics}-{term_type}.html"
	pyLDAvis.save_html(vis_data, output)

In [96]:
## HDP (max_n_topics = 45)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 45
hdp = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics)
vis_data = pyLDAvis.gensim.prepare(hdp, corpus, diction)
pyLDAvis.display(vis_data)

In [97]:
## HDP (max_n_topics = 15)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 15
hdp = gensim.models.HdpModel(corpus, diction, random_state = 1, T = max_n_topics)
vis_data = pyLDAvis.gensim.prepare(hdp, corpus, diction)
pyLDAvis.display(vis_data)